#  [lenet01] Tutorial: Image classification - Mnist

在這個教學裡面，會教你怎麼透過模組化的 dl 框架來完成訓練。

In [1]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

## Datasets

首先準備 mnist 資料。

In [2]:
import torch
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, ToPILImage

train_dataset = MNIST('/files/', train=True, download=True, transform=ToTensor())
test_dataset = MNIST('/files/', train=False, download=True, transform=ToTensor())

train_dataset, valid_dataset = random_split(train_dataset, [0.9, 0.1], generator=torch.Generator().manual_seed(42))

print("There are", len(train_dataset), "images in the training data.")
print("There are", len(valid_dataset), "images in the validation data.")
print("There are", len(test_dataset), "images in the testing data.")

There are 54000 images in the training data.
There are 6000 images in the validation data.
There are 10000 images in the testing data.


In [3]:
first_image = train_dataset[0][0]
print("Shape of input:", first_image.shape)

first_image = ToPILImage()(first_image)
first_target = train_dataset[0][1]
display(first_image)
print("First target:", first_target)

Shape of input: torch.Size([1, 28, 28])


First target: 6


In [4]:
# DataLoaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataloader = DataLoader(valid_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

## Neural networks

整個訓練 + 測試的流程涉及以下三個類別(Class)

* Trainer: 用來管理訓練流程
* Validator: 用來計算驗證、測試的指標
* Updater: 用來控制更新神經網路的單步 (E.g. 一次反向傳播)

我們只需要在原有的 nn.Module 加上一些屬性(例如 optimizer 和 criterion) <br>
並製作一個計算驗證指標的函式讓 Validator 使用，<br>
就可以開始訓練了！


In [5]:
import torch
import torch.nn as nn
from pathlib import Path

class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5, 5), padding=2),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=(2, 2), stride=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5)),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=(2, 2), stride=2),
            nn.Flatten(),
            nn.Linear(in_features=5*5*16, out_features=120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(in_features=84, out_features=10)
        )
        
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=0.001) # called by the updater
        self.criterion = nn.CrossEntropyLoss()   # called by the updater

    def forward(self, x):
        return self.net(x)

    def inference(self, x):
        # called by the validator
        preds = self.forward(x)
        pred_labels = torch.argmax(preds, dim=1)
        return pred_labels
    
    def save(self, checkpoint_dir):
        # called by the validator
        Path(checkpoint_dir).mkdir(parents=True, exist_ok=True)
        torch.save(self.net.state_dict(), str(Path(checkpoint_dir) / "net.pth"))
    
    def load(self, checkpoint_dir):
        # called by the validator
        self.net.load_state_dict(torch.load(str(Path(checkpoint_dir) / "net.pth")))


def batch_acc(pred_labels, targets):
    # called by the validator
    correct = (pred_labels == targets).sum().item()
    total = len(targets)
    return torch.tensor(correct / total)

In [6]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from modules.base.trainer import BaseTrainer
from modules.base.updater import BaseUpdater
from modules.base.validator import BaseValidator

validator = BaseValidator(metric=batch_acc)
updater = BaseUpdater()
trainer = BaseTrainer(max_iter=1000, eval_step=334, validator=validator)

# train
print("Train:")
lenet = LeNet5().cuda()
trainer.train(module=lenet, updater=updater, train_dataloader=train_dataloader, val_dataloader=val_dataloader)

# test
print("\n Test:")
validator.validation(module=lenet, dataloader=test_dataloader)

Train:
--------
Device: cuda
# of Training Samples: 422
# of Validation Samples: 47
Max iteration: 1000 steps (validates per 334 steps)
Checkpoint directory: ./checkpoints/
Evaluation metric: function
--------


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

2024-08-17 05:20:43.806 | SUCCESS  | modules.base.trainer:success:71 - Model saved! Validation: (New) 0.75404 > (Old) 0.00000


  0%|          | 0/47 [00:00<?, ?it/s]

2024-08-17 05:20:51.297 | SUCCESS  | modules.base.trainer:success:71 - Model saved! Validation: (New) 0.88944 > (Old) 0.75404


  0%|          | 0/47 [00:00<?, ?it/s]

2024-08-17 05:20:58.919 | SUCCESS  | modules.base.trainer:success:71 - Model saved! Validation: (New) 0.91784 > (Old) 0.88944

 Test:


  0%|          | 0/79 [00:00<?, ?it/s]

0.9236550632911392

搞定！